In [22]:
import tensorflow as tf

from keras.layers import Input, Dense, LeakyReLU, Dropout, BatchNormalization
from keras.models import Model
from keras.optimizers import SGD, Adam
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os

tf.autograph.set_verbosity(0)

tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [23]:
data = []

for path in os.listdir("O:\Desktop\Learning Machine Learning\Machine-Learning-Portfolio\kaggle comps\painter\images"):
    img_array = cv2.imread(f'O:\Desktop\Learning Machine Learning\Machine-Learning-Portfolio\kaggle comps\painter\images\{path}')
    rgb_img = cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB)
    data.append(rgb_img)

In [24]:
xtrain = np.stack(data[:int(0.7*len(data))], axis=0)
xtest = np.stack(data[int(0.7*len(data))+1:], axis=0)

In [25]:
# Flatten data

xtrain, xtest = xtrain / 255.0 * 2 - 1, xtest / 255.0 * 2 - 1

N, H, W, C = xtrain.shape
print(xtrain.shape)
D = H * W * C
xtrain = xtrain.reshape(-1, D)
xtest = xtest.reshape(-1, D)

(210, 256, 256, 3)


In [26]:
latent_dim = 100

In [27]:
# Build Generator
# latent dim is a hyperparameter

def build_gen(latent_dim):
    i = Input(shape=(latent_dim,))
    x = Dense(256, activation=LeakyReLU(alpha=0.2))(i)
    x = BatchNormalization(momentum=0.8)(x)
    x = Dense(512, activation=LeakyReLU(alpha=0.2))(x)
    x = BatchNormalization(momentum=0.8)(x)
    x = Dense(1024, activation=LeakyReLU(alpha=0.2))(x)
    x = BatchNormalization(momentum=0.8)(x)
    x = Dense(D, activation='tanh')(x)
    model = Model(i, x)
    return model

In [28]:
# Build the discriminator
def build_disc(img_size):
    i = Input(shape=(img_size,))
    x = Dense(512, activation=LeakyReLU(alpha=0.2))(i)
    x = Dense(256, activation=LeakyReLU(alpha=0.2))(x)
    x = Dense(1, activation="sigmoid")(x)
    model = Model(i, x)
    return model

In [30]:
# compile the models

# making the discriminator
disc = build_disc(D)
disc.compile(
    loss='binary_crossentropy',
    optimizer=Adam(0.0002, 0.5),
    metrics=["accuracy"]
)
# making the generator
gen = build_gen(latent_dim)

# input to represent the noise sample from the latent space
z = Input(shape=(latent_dim,))

# generate an image from the noise
img = gen(z)

# freeze the discriminators learning
disc.trainable = False

fakepred = disc(img)

combinedModel = Model(z, fakepred)

combinedModel.compile(
    loss='binary_crossentropy',
    optimizer=Adam(0.0002, 0.5),
)

ResourceExhaustedError: {{function_node __wrapped__StatelessRandomUniformV2_device_/job:localhost/replica:0/task:0/device:GPU:0}} OOM when allocating tensor with shape[196608,512] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:StatelessRandomUniformV2]

In [ ]:
# training

batch_size = 32
epochs = 30000
sample_period = 100

ones = np.ones(batch_size)
zeros = np.zeros(batch_size)

d_losses = []
g_losses = []



In [ ]:
def sample_images(epoch):
    rows, colums = 5, 5
    noise = np.random.rand(latent_dim)
    img = gen.predict(noise)
    img = 0.5 * img + 0.5
    plt.plot(img)
    plt.savefig(f"GAN images/{epoch}")
    plt.close()

In [ ]:
# training 

for epoch in range(epochs):

    idx = np.random.randint(0, xtrain.shape[0], batch_size)
    real_imgs = xtrain[idx]

    noise = np.random.rand(batch_size, latent_dim)
    fake_imgs = gen.predict(noise)

    d_loss_real, d_acc_real = disc.train_on_batch(real_imgs, ones)
    d_loss_fake, d_acc_fake = disc.train_on_batch(fake_imgs, zeros)

    d_loss = 0.5 * (d_loss_real + d_loss_fake)
    d_acc = 0.5 * (d_acc_real + d_acc_fake)

    # train generator

    noise = np.random.rand(batch_size, latent_dim)
    g_loss = combinedModel.train_on_batch(noise, ones)

    d_losses.append(d_loss)
    g_losses.append(g_loss)

    if epoch % 100 == 0:
        print(f"epoch:{epoch+1}/{epochs}, d_loss:{d_loss:.2f}, d_acc:{d_acc:.2f}, g_loss:{g_loss:.2f} ")

    if epoch % sample_period == 0:
        sample_images(epoch)

1/1 [==============================] - 2s 2s/step
epoch:1/10000, d_loss:0.57, d_acc:0.59, g_loss:0.64 
1/1 [==============================] - 0s 15ms/step
epoch:101/10000, d_loss:0.60, d_acc:0.88, g_loss:10.23 
1/1 [==============================] - 0s 15ms/step
epoch:201/10000, d_loss:1.29, d_acc:0.83, g_loss:11.14 
1/1 [==============================] - 0s 17ms/step
epoch:301/10000, d_loss:0.32, d_acc:0.94, g_loss:10.65 
1/1 [==============================] - 0s 15ms/step
epoch:401/10000, d_loss:1.36, d_acc:0.73, g_loss:14.34 
1/1 [==============================] - 0s 14ms/step
epoch:501/10000, d_loss:1.35, d_acc:0.77, g_loss:15.13 
1/1 [==============================] - 0s 14ms/step
epoch:601/10000, d_loss:1.03, d_acc:0.81, g_loss:7.42 
1/1 [==============================] - 0s 15ms/step
epoch:701/10000, d_loss:0.42, d_acc:0.86, g_loss:15.57 
1/1 [==============================] - 0s 15ms/step
epoch:801/10000, d_loss:0.71, d_acc:0.94, g_loss:14.00 
1/1 [=============================

KeyboardInterrupt: 